In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.preprocessing import OrdinalEncoder # for encoding labels
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight

#from imblearn.over_sampling import SMOTE
from collections import Counter
import io
import requests
from sklearn import metrics

from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.calibration import CalibrationDisplay
from sklearn.metrics import brier_score_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense # for creating regular densely-connected NN layer.
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout # for adding Concolutional and densely-connected NN layers.
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU as LR,\
                                    Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Conv2D,MaxPooling2D,Dense,Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from keras import callbacks
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense # for creating regular densely-connected NN layer.
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,MaxPooling2D # for adding Concolutional and densely-connected NN layers.
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Cropping2D


2024-01-02 22:17:35.193427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    c10 = Cropping2D(cropping=((1,2),(1,2)))(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c10)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

### Indices

In [2]:
path = ('/home/b/b382633/TC_adds/indices/')
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.read_csv(all_files[0])
for f in all_files[1:]:
    aux_df = pd.read_csv(f)
    df = pd.concat((df,aux_df.iloc[:,1:]), axis=1)
df

,Unnamed: 0,PNA,NPD,weu,NAOP,BLOCK,NAOM,RIDGE,NAOP_S,BLOCK_S,...,weu.2,NAO_classic,PNA,NPD,Arctic,glob,30_40N,EOF1,EOF2,EOF3
0,1980-01-01,6.380449,0.070947,0.376710,-0.698796,-1.091872,0.916174,0.858326,-0.182057,-1.804146,...,0.364284,5.982506,6.380449,0.070947,90.527814,11.746197,7.888408,-1.255583,-0.922094,0.257785
1,1980-01-02,4.327160,-0.372845,0.376665,-0.541367,-0.628396,0.457480,0.697619,-0.333924,-0.904377,...,0.364769,16.567497,4.327160,-0.372845,87.659024,11.693716,7.836839,-0.879452,-0.530689,-0.097658
2,1980-01-03,0.487942,-1.042550,0.376157,-0.415616,-0.114731,0.089294,0.449448,-0.382689,0.013145,...,0.365231,28.132500,0.487942,-1.042550,66.456917,11.644454,7.729588,-0.493031,-0.065794,-0.279278
3,1980-01-04,-2.873871,1.342918,0.375406,-0.293791,-0.132405,0.184506,0.257954,-0.072198,-0.015091,...,0.365614,36.470001,-2.873871,1.342918,63.375784,11.504785,6.802492,-0.346260,-0.094309,0.001761
4,1980-01-05,-4.641118,5.615116,0.374966,-0.359942,0.191257,0.242473,0.002844,-0.261882,0.014397,...,0.365948,21.022499,-4.641118,5.615116,70.221382,11.388587,6.401954,-0.219013,0.364039,0.311780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,1.416902,4.614209,0.241504,-0.071530,0.075940,-0.405470,0.356717,-0.560067,0.487704,...,0.317727,20.385002,1.416902,4.614209,-29.761222,16.321686,23.101869,-0.080529,-0.056509,-0.774756
15946,2023-08-29,2.652396,3.251957,0.244578,-0.100789,-0.055382,-0.319646,0.424297,-0.534366,0.219864,...,0.317635,10.308746,2.652396,3.251957,-16.153422,16.326821,23.115247,-0.183080,-0.123778,-0.712566
15947,2023-08-30,1.843450,0.805065,0.246882,0.030442,0.005388,-0.279854,0.204001,-0.241968,0.276627,...,0.317551,4.352501,1.843450,0.805065,-24.041468,16.317558,23.257136,-0.007641,-0.063605,-0.420079
15948,2023-08-31,-1.793373,-0.245677,0.247781,0.084769,0.044327,-0.168327,0.019747,-0.029180,0.304703,...,0.317450,5.427505,-1.793373,-0.245677,-21.431139,16.189238,23.093952,0.094012,-0.007238,-0.095871


In [4]:
# WeMO index removed since it has some Nan and time renamed
df = df.dropna(axis=1).rename(columns={'Unnamed: 0':'time'})
df

,time,PNA,NPD,weu,NAOP,BLOCK,NAOM,RIDGE,NAOP_S,BLOCK_S,...,weu.2,NAO_classic,PNA,NPD,Arctic,glob,30_40N,EOF1,EOF2,EOF3
0,1980-01-01,6.380449,0.070947,0.376710,-0.698796,-1.091872,0.916174,0.858326,-0.182057,-1.804146,...,0.364284,5.982506,6.380449,0.070947,90.527814,11.746197,7.888408,-1.255583,-0.922094,0.257785
1,1980-01-02,4.327160,-0.372845,0.376665,-0.541367,-0.628396,0.457480,0.697619,-0.333924,-0.904377,...,0.364769,16.567497,4.327160,-0.372845,87.659024,11.693716,7.836839,-0.879452,-0.530689,-0.097658
2,1980-01-03,0.487942,-1.042550,0.376157,-0.415616,-0.114731,0.089294,0.449448,-0.382689,0.013145,...,0.365231,28.132500,0.487942,-1.042550,66.456917,11.644454,7.729588,-0.493031,-0.065794,-0.279278
3,1980-01-04,-2.873871,1.342918,0.375406,-0.293791,-0.132405,0.184506,0.257954,-0.072198,-0.015091,...,0.365614,36.470001,-2.873871,1.342918,63.375784,11.504785,6.802492,-0.346260,-0.094309,0.001761
4,1980-01-05,-4.641118,5.615116,0.374966,-0.359942,0.191257,0.242473,0.002844,-0.261882,0.014397,...,0.365948,21.022499,-4.641118,5.615116,70.221382,11.388587,6.401954,-0.219013,0.364039,0.311780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,1.416902,4.614209,0.241504,-0.071530,0.075940,-0.405470,0.356717,-0.560067,0.487704,...,0.317727,20.385002,1.416902,4.614209,-29.761222,16.321686,23.101869,-0.080529,-0.056509,-0.774756
15946,2023-08-29,2.652396,3.251957,0.244578,-0.100789,-0.055382,-0.319646,0.424297,-0.534366,0.219864,...,0.317635,10.308746,2.652396,3.251957,-16.153422,16.326821,23.115247,-0.183080,-0.123778,-0.712566
15947,2023-08-30,1.843450,0.805065,0.246882,0.030442,0.005388,-0.279854,0.204001,-0.241968,0.276627,...,0.317551,4.352501,1.843450,0.805065,-24.041468,16.317558,23.257136,-0.007641,-0.063605,-0.420079
15948,2023-08-31,-1.793373,-0.245677,0.247781,0.084769,0.044327,-0.168327,0.019747,-0.029180,0.304703,...,0.317450,5.427505,-1.793373,-0.245677,-21.431139,16.189238,23.093952,0.094012,-0.007238,-0.095871


In [22]:
# Save as unique file
df.to_csv('/Users/paolo/Documents/TC/Paolo/data/Indices/full_indices.csv',index=False)

In [12]:
df = pd.read_csv('/home/b/b382633/TC_adds/indices/full_indices.csv')
df.loc[df.time<='2020-12-31']

,time,NAOP_S,BLOCK_S,NAOM_S,RIDGE_S,nino3,nino4,nino34,nino12,indocW,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,1980-01-01,-0.182057,-1.804146,0.690041,1.425914,23.954093,28.309878,26.591961,24.046144,28.036842,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1980-01-02,-0.333924,-0.904377,0.195294,1.178619,23.982840,28.305696,26.569124,24.003577,28.076732,...,-0.541367,-0.628396,0.457480,0.697619,-4.316831,31.318435,0.364769,16.567497,4.327160,-0.372845
2,1980-01-03,-0.382689,0.013145,-0.074517,0.502674,24.053181,28.349518,26.608834,23.893645,28.089537,...,-0.415616,-0.114731,0.089294,0.449448,-4.096059,9.162693,0.365231,28.132500,0.487942,-1.042550
3,1980-01-04,-0.072198,-0.015091,0.035592,0.045248,24.127478,28.406776,26.680904,23.756035,28.055619,...,-0.293791,-0.132405,0.184506,0.257954,-4.563278,8.043160,0.365614,36.470001,-2.873871,1.342918
4,1980-01-05,-0.261882,0.014397,0.114259,0.117818,24.184204,28.460211,26.768365,23.657757,27.981568,...,-0.359942,0.191257,0.242473,0.002844,-6.212076,25.560463,0.365948,21.022499,-4.641118,5.615116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14971,2020-12-27,0.479521,-2.190530,1.307611,0.346930,23.737911,27.397964,25.525410,22.278842,28.250424,...,-0.234060,-1.764493,0.857381,0.995587,-7.548152,90.372605,0.325706,44.727501,0.404606,-1.170120
14972,2020-12-28,-0.788696,-1.967568,2.106056,0.450566,23.823000,27.367803,25.521198,22.368278,28.333251,...,-0.946460,-1.317045,1.229010,1.032071,-7.459093,114.348871,0.326303,-2.437500,-1.989788,1.100273
14973,2020-12-29,-1.035601,-0.710435,1.390727,0.177524,23.959376,27.330603,25.517207,22.426340,28.321008,...,-0.854235,-0.601319,0.731468,0.758283,-7.443579,95.472285,0.326973,-6.042496,-1.261597,1.695706
14974,2020-12-30,-0.934227,-0.685784,1.265412,0.196886,24.065765,27.295215,25.529809,22.389923,28.276813,...,-0.818814,-0.464725,0.704163,0.630745,-8.496942,96.203054,0.327704,10.727501,0.213722,0.165452


### Add as additional columns of Sindian data

In [8]:
df = pd.read_csv('/home/b/b382633/TC_adds/indices/full_indices.csv')
df

,time,NAOP_S,BLOCK_S,NAOM_S,RIDGE_S,nino3,nino4,nino34,nino12,indocW,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,1980-01-01,-0.182057,-1.804146,0.690041,1.425914,23.954093,28.309878,26.591961,24.046144,28.036842,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1980-01-02,-0.333924,-0.904377,0.195294,1.178619,23.982840,28.305696,26.569124,24.003577,28.076732,...,-0.541367,-0.628396,0.457480,0.697619,-4.316831,31.318435,0.364769,16.567497,4.327160,-0.372845
2,1980-01-03,-0.382689,0.013145,-0.074517,0.502674,24.053181,28.349518,26.608834,23.893645,28.089537,...,-0.415616,-0.114731,0.089294,0.449448,-4.096059,9.162693,0.365231,28.132500,0.487942,-1.042550
3,1980-01-04,-0.072198,-0.015091,0.035592,0.045248,24.127478,28.406776,26.680904,23.756035,28.055619,...,-0.293791,-0.132405,0.184506,0.257954,-4.563278,8.043160,0.365614,36.470001,-2.873871,1.342918
4,1980-01-05,-0.261882,0.014397,0.114259,0.117818,24.184204,28.460211,26.768365,23.657757,27.981568,...,-0.359942,0.191257,0.242473,0.002844,-6.212076,25.560463,0.365948,21.022499,-4.641118,5.615116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-0.560067,0.487704,-0.277526,0.405942,26.790646,29.710604,28.244773,24.355470,28.360921,...,-0.071530,0.075940,-0.405470,0.356717,1.888322,13.145353,0.317727,20.385002,1.416902,4.614209
15946,2023-08-29,-0.534366,0.219864,-0.234510,0.637364,26.755699,29.741776,28.258911,24.340400,28.343777,...,-0.100789,-0.055382,-0.319646,0.424297,1.590642,20.408620,0.317635,10.308746,2.652396,3.251957
15947,2023-08-30,-0.241968,0.276627,-0.285007,0.311772,26.657818,29.795078,28.304199,24.270261,28.330869,...,0.030442,0.005388,-0.279854,0.204001,1.038108,9.354852,0.317551,4.352501,1.843450,0.805065
15948,2023-08-31,-0.029180,0.304703,-0.163869,-0.114591,26.499222,29.792601,28.270638,24.195576,28.241875,...,0.084769,0.044327,-0.168327,0.019747,-0.953342,7.679418,0.317450,5.427505,-1.793373,-0.245677


In [9]:
train = pd.read_csv('/home/b/b382633/TC_adds/csv_extracted/train_dailymeans_Sindian.csv')
train

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,sst,tcwv,tclw,tciw,shear
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,0.00000,45.009384,0.104688,0.023254,6.302022
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,0.00000,43.669964,0.104326,0.007427,5.039517
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,0.00000,44.030956,0.097731,0.044125,5.275516
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,0.00000,39.961853,0.096560,0.010863,7.474898
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,0.00000,28.731860,0.073104,0.007928,8.653576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,296.89227,26.542997,0.030980,0.000126,28.859175
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,296.03314,23.791655,0.030671,0.000000,28.086445
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,295.36078,23.877638,0.037318,0.000000,29.294120
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,295.10638,22.520674,0.057125,0.000000,30.179830


In [10]:
train_withIndex = train.merge(df, on='time', how='left')
train_withIndex

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247


In [11]:
train_withIndex.to_csv('/home/b/b382633/TC_adds/indices/train_dailymeans_Sindian.csv',index=False)


In [12]:
val = pd.read_csv('/home/b/b382633/TC_adds/csv_extracted/val_dailymeans_Sindian.csv')
val

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,sst,tcwv,tclw,tciw,shear
0,0,0.0,20.0,2011-01-01,0.000003,73.016390,-5.760780,-4.216808,6.860649,-4.352928,0.00000,45.908466,0.051140,0.010802,11.319371
1,1,0.0,22.5,2011-01-01,0.000003,74.569660,-4.942451,-3.857407,6.459419,-3.991157,0.00000,45.768530,0.054900,0.014612,10.506754
2,2,0.0,25.0,2011-01-01,0.000004,80.080090,-3.848740,-3.175144,6.303680,-3.446140,0.00000,47.055485,0.085401,0.041099,9.773062
3,3,0.0,27.5,2011-01-01,0.000012,83.676704,0.330811,-2.526569,7.235268,-2.307594,0.00000,42.947980,0.091923,0.081013,9.961468
4,4,0.0,30.0,2011-01-01,0.000011,76.225440,3.678749,-1.027561,7.020271,-0.077572,0.00000,32.404747,0.098393,0.042447,8.516380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688397,688397,-30.0,80.0,2015-12-31,0.000014,70.662056,23.560066,1.655861,9.690376,3.621418,296.77530,33.177513,0.011262,0.000468,22.729418
688398,688398,-30.0,82.5,2015-12-31,-0.000006,51.213654,22.381706,0.321705,9.860390,-0.099480,296.44290,30.789354,0.029158,0.000318,24.204187
688399,688399,-30.0,85.0,2015-12-31,0.000009,46.678970,22.464828,0.851299,7.661758,-0.725330,295.73486,30.004541,0.039861,0.000132,23.183786
688400,688400,-30.0,87.5,2015-12-31,0.000002,59.362090,22.364807,0.543045,5.595253,-1.542034,295.24792,31.301584,0.131010,0.001035,22.959316


In [13]:
val_withIndex = val.merge(df, on='time', how='left')
val_withIndex

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,0,0.0,20.0,2011-01-01,0.000003,73.016390,-5.760780,-4.216808,6.860649,-4.352928,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
1,1,0.0,22.5,2011-01-01,0.000003,74.569660,-4.942451,-3.857407,6.459419,-3.991157,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
2,2,0.0,25.0,2011-01-01,0.000004,80.080090,-3.848740,-3.175144,6.303680,-3.446140,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
3,3,0.0,27.5,2011-01-01,0.000012,83.676704,0.330811,-2.526569,7.235268,-2.307594,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
4,4,0.0,30.0,2011-01-01,0.000011,76.225440,3.678749,-1.027561,7.020271,-0.077572,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688397,688397,-30.0,80.0,2015-12-31,0.000014,70.662056,23.560066,1.655861,9.690376,3.621418,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407
688398,688398,-30.0,82.5,2015-12-31,-0.000006,51.213654,22.381706,0.321705,9.860390,-0.099480,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407
688399,688399,-30.0,85.0,2015-12-31,0.000009,46.678970,22.464828,0.851299,7.661758,-0.725330,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407
688400,688400,-30.0,87.5,2015-12-31,0.000002,59.362090,22.364807,0.543045,5.595253,-1.542034,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407


In [14]:
val_withIndex.to_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv',index=False)


In [15]:
test = pd.read_csv('/home/b/b382633/TC_adds/csv_extracted/test_dailymeans_Sindian.csv')
test

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,sst,tcwv,tclw,tciw,shear
0,0,0.0,20.0,2016-04-01,0.000004,81.055560,-8.377182,5.273445,2.527817,2.160847,0.00000,53.959347,0.066320,0.052278,13.655559
1,1,0.0,22.5,2016-04-01,-0.000002,76.221700,-6.143539,4.284462,4.278809,2.030232,0.00000,53.494244,0.086551,0.236994,10.667676
2,2,0.0,25.0,2016-04-01,0.000018,75.465416,-1.724731,3.315552,3.811935,1.344738,0.00000,53.025135,0.314800,0.518474,5.611731
3,3,0.0,27.5,2016-04-01,-0.000009,72.850200,1.577347,1.549385,4.243774,2.951813,0.00000,48.393536,0.186379,0.423175,1.292264
4,4,0.0,30.0,2016-04-01,-0.000009,82.229460,1.983276,1.535709,5.736717,0.056183,0.00000,41.883312,0.094952,0.209450,5.698139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929677,929677,-30.0,80.0,2022-12-31,0.000008,14.955483,13.163338,-4.289364,4.153252,-2.868565,295.33643,24.162498,0.033585,0.001839,18.812302
929678,929678,-30.0,82.5,2022-12-31,0.000023,10.061256,13.957420,-4.323608,7.508942,-0.484577,295.08510,23.452278,0.044212,0.001561,19.952251
929679,929679,-30.0,85.0,2022-12-31,0.000009,8.901871,15.958916,-3.855770,11.119324,2.067028,294.75940,19.158775,0.035525,0.000474,21.784533
929680,929680,-30.0,87.5,2022-12-31,-0.000002,8.630913,19.073456,-3.618012,14.113441,2.629562,294.41754,17.160767,0.043179,0.001137,25.431913


In [16]:
test_withIndex = test.merge(df, on='time', how='left')
test_withIndex

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,0,0.0,20.0,2016-04-01,0.000004,81.055560,-8.377182,5.273445,2.527817,2.160847,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
1,1,0.0,22.5,2016-04-01,-0.000002,76.221700,-6.143539,4.284462,4.278809,2.030232,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
2,2,0.0,25.0,2016-04-01,0.000018,75.465416,-1.724731,3.315552,3.811935,1.344738,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
3,3,0.0,27.5,2016-04-01,-0.000009,72.850200,1.577347,1.549385,4.243774,2.951813,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
4,4,0.0,30.0,2016-04-01,-0.000009,82.229460,1.983276,1.535709,5.736717,0.056183,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929677,929677,-30.0,80.0,2022-12-31,0.000008,14.955483,13.163338,-4.289364,4.153252,-2.868565,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011
929678,929678,-30.0,82.5,2022-12-31,0.000023,10.061256,13.957420,-4.323608,7.508942,-0.484577,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011
929679,929679,-30.0,85.0,2022-12-31,0.000009,8.901871,15.958916,-3.855770,11.119324,2.067028,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011
929680,929680,-30.0,87.5,2022-12-31,-0.000002,8.630913,19.073456,-3.618012,14.113441,2.629562,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011


In [17]:
test_withIndex.to_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv',index=False)


In [18]:
df.columns

Index(['time', 'NAOP_S', 'BLOCK_S', 'NAOM_S', 'RIDGE_S', 'nino3', 'nino4',
       'nino34', 'nino12', 'indocW', 'indocE', 'AtlMDR', 'CarMDR', 'GulfMDR',
       'AtlSub', 'glob', '30_40N', 'weu', 'EOF1', 'EOF2', 'EOF3', 'Arctic',
       'eq', '60N', '60S', 'weu.1', 'eeu', 'cus', 'NAO', 'EAT', 'ABLOCK',
       'NAOP', 'BLOCK', 'NAOM', 'RIDGE', 'EAS', 'Arctic.1', 'weu.2',
       'NAO_classic', 'PNA', 'NPD'],
      dtype='object')

### PCA

In [78]:
df = pd.read_csv('/home/b/b382633/TC_adds/indices/full_indices.csv')

df

,time,NAOP_S,BLOCK_S,NAOM_S,RIDGE_S,nino3,nino4,nino34,nino12,indocW,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,1980-01-01,-0.182057,-1.804146,0.690041,1.425914,23.954093,28.309878,26.591961,24.046144,28.036842,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1980-01-02,-0.333924,-0.904377,0.195294,1.178619,23.982840,28.305696,26.569124,24.003577,28.076732,...,-0.541367,-0.628396,0.457480,0.697619,-4.316831,31.318435,0.364769,16.567497,4.327160,-0.372845
2,1980-01-03,-0.382689,0.013145,-0.074517,0.502674,24.053181,28.349518,26.608834,23.893645,28.089537,...,-0.415616,-0.114731,0.089294,0.449448,-4.096059,9.162693,0.365231,28.132500,0.487942,-1.042550
3,1980-01-04,-0.072198,-0.015091,0.035592,0.045248,24.127478,28.406776,26.680904,23.756035,28.055619,...,-0.293791,-0.132405,0.184506,0.257954,-4.563278,8.043160,0.365614,36.470001,-2.873871,1.342918
4,1980-01-05,-0.261882,0.014397,0.114259,0.117818,24.184204,28.460211,26.768365,23.657757,27.981568,...,-0.359942,0.191257,0.242473,0.002844,-6.212076,25.560463,0.365948,21.022499,-4.641118,5.615116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-0.560067,0.487704,-0.277526,0.405942,26.790646,29.710604,28.244773,24.355470,28.360921,...,-0.071530,0.075940,-0.405470,0.356717,1.888322,13.145353,0.317727,20.385002,1.416902,4.614209
15946,2023-08-29,-0.534366,0.219864,-0.234510,0.637364,26.755699,29.741776,28.258911,24.340400,28.343777,...,-0.100789,-0.055382,-0.319646,0.424297,1.590642,20.408620,0.317635,10.308746,2.652396,3.251957
15947,2023-08-30,-0.241968,0.276627,-0.285007,0.311772,26.657818,29.795078,28.304199,24.270261,28.330869,...,0.030442,0.005388,-0.279854,0.204001,1.038108,9.354852,0.317551,4.352501,1.843450,0.805065
15948,2023-08-31,-0.029180,0.304703,-0.163869,-0.114591,26.499222,29.792601,28.270638,24.195576,28.241875,...,0.084769,0.044327,-0.168327,0.019747,-0.953342,7.679418,0.317450,5.427505,-1.793373,-0.245677


In [79]:
from sklearn.decomposition import PCA
import numpy as np
pca = PCA(n_components=0.99)
red_X_99 = pca.fit_transform(df.iloc[:,1:])

names = ['time']
for i in range(red_X_99.shape[1]):
    names.append('comp99_'+str(i))
names

red_df_99 = pd.DataFrame(np.concatenate((df.time.values.reshape(-1,1),red_X_99),axis=1),columns=names)#.rename(columns={0:'time'})
red_df_99

,time,comp99_0,comp99_1,comp99_2,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7
0,1980-01-01,83.806267,60.81163,-37.711484,-22.522648,-17.040834,-4.716553,-5.327079,4.680688
1,1980-01-02,74.552681,62.81259,-34.29683,-22.060617,-26.121479,-3.582355,-4.297971,1.700701
2,1980-01-03,42.879099,62.979958,-26.535009,-21.568343,-30.888566,-3.567491,-1.976986,-1.825118
3,1980-01-04,39.239566,64.712265,-20.704652,-20.293211,-36.299514,-1.978507,1.755396,-2.470499
4,1980-01-05,59.07532,63.086554,-23.310044,-18.588885,-23.402192,-5.655348,6.397826,-3.037748
...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-2.690358,-53.624069,31.383152,27.083869,-1.433984,-10.55502,0.699701,4.365854
15946,2023-08-29,12.19268,-53.413492,21.55142,25.770639,1.572338,-13.19754,-0.713921,4.022209
15947,2023-08-30,-0.572852,-56.304293,17.369056,26.60048,6.799272,-14.412565,-1.2231,2.077897
15948,2023-08-31,-0.360282,-57.69965,15.826201,25.263325,3.404456,-15.439204,0.600153,-0.812687


In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
red_X_95 = pca.fit_transform(df.iloc[:,1:])

names = ['time']
for i in range(red_X_95.shape[1]):
    names.append('comp95_'+str(i))
names

red_df_95 = pd.DataFrame(np.concatenate((df.time.values.reshape(-1,1),red_X_95),axis=1),columns=names)#.rename(columns={0:'time'})
red_df_95

,time,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,1980-01-01,83.806267,60.81163,-37.711484,-22.522648,-17.040834
1,1980-01-02,74.552681,62.81259,-34.29683,-22.060617,-26.121479
2,1980-01-03,42.879099,62.979958,-26.535009,-21.568343,-30.888566
3,1980-01-04,39.239566,64.712265,-20.704652,-20.293211,-36.299514
4,1980-01-05,59.07532,63.086554,-23.310044,-18.588885,-23.402192
...,...,...,...,...,...,...
15945,2023-08-28,-2.690358,-53.624069,31.383152,27.083869,-1.433984
15946,2023-08-29,12.19268,-53.413492,21.55142,25.770639,1.572338
15947,2023-08-30,-0.572852,-56.304293,17.369056,26.60048,6.799272
15948,2023-08-31,-0.360282,-57.69965,15.826201,25.263325,3.404456


In [6]:
red_df_95.to_csv('/home/b/b382633/TC_adds/indices/red_df_95.csv',index=False)

In [80]:
red_df_99.to_csv('/home/b/b382633/TC_adds/indices/red_df_99.csv',index=False)

In [31]:
train = pd.read_csv('/home/b/b382633/TC_adds/indices/train_dailymeans_Sindian.csv')

train_withIndexPCA = train.merge(red_df_99, on='time', how='left')
train_withIndexPCA = train_withIndexPCA.merge(red_df_95, on='time', how='left')

train_withIndexPCA

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.81163,-37.711484,-22.522648,-17.040834
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.81163,-37.711484,-22.522648,-17.040834
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.81163,-37.711484,-22.522648,-17.040834
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.81163,-37.711484,-22.522648,-17.040834
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.81163,-37.711484,-22.522648,-17.040834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671


In [32]:
train_withIndexPCA.to_csv('/home/b/b382633/TC_adds/indices/train_dailymeans_Sindian.csv',index=False)


In [33]:
val = pd.read_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv')

val_withIndexPCA = val.merge(red_df_99, on='time', how='left')
val_withIndexPCA = val_withIndexPCA.merge(red_df_95, on='time', how='left')

val_withIndexPCA


,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,0,0.0,20.0,2011-01-01,0.000003,73.016390,-5.760780,-4.216808,6.860649,-4.352928,...,-4.164251,26.392069,-18.645216,0.040749,-3.512868,85.353787,57.763623,-4.800264,-4.164251,26.392069
1,1,0.0,22.5,2011-01-01,0.000003,74.569660,-4.942451,-3.857407,6.459419,-3.991157,...,-4.164251,26.392069,-18.645216,0.040749,-3.512868,85.353787,57.763623,-4.800264,-4.164251,26.392069
2,2,0.0,25.0,2011-01-01,0.000004,80.080090,-3.848740,-3.175144,6.303680,-3.446140,...,-4.164251,26.392069,-18.645216,0.040749,-3.512868,85.353787,57.763623,-4.800264,-4.164251,26.392069
3,3,0.0,27.5,2011-01-01,0.000012,83.676704,0.330811,-2.526569,7.235268,-2.307594,...,-4.164251,26.392069,-18.645216,0.040749,-3.512868,85.353787,57.763623,-4.800264,-4.164251,26.392069
4,4,0.0,30.0,2011-01-01,0.000011,76.225440,3.678749,-1.027561,7.020271,-0.077572,...,-4.164251,26.392069,-18.645216,0.040749,-3.512868,85.353787,57.763623,-4.800264,-4.164251,26.392069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688397,688397,-30.0,80.0,2015-12-31,0.000014,70.662056,23.560066,1.655861,9.690376,3.621418,...,19.63736,8.555269,-17.270908,-4.81614,-2.152159,-9.787334,65.429647,92.772204,19.63736,8.555269
688398,688398,-30.0,82.5,2015-12-31,-0.000006,51.213654,22.381706,0.321705,9.860390,-0.099480,...,19.63736,8.555269,-17.270908,-4.81614,-2.152159,-9.787334,65.429647,92.772204,19.63736,8.555269
688399,688399,-30.0,85.0,2015-12-31,0.000009,46.678970,22.464828,0.851299,7.661758,-0.725330,...,19.63736,8.555269,-17.270908,-4.81614,-2.152159,-9.787334,65.429647,92.772204,19.63736,8.555269
688400,688400,-30.0,87.5,2015-12-31,0.000002,59.362090,22.364807,0.543045,5.595253,-1.542034,...,19.63736,8.555269,-17.270908,-4.81614,-2.152159,-9.787334,65.429647,92.772204,19.63736,8.555269


In [34]:
val_withIndexPCA.to_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv',index=False)


In [35]:
test = pd.read_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv')

test_withIndexPCA = test.merge(red_df_99, on='time', how='left')
test_withIndexPCA = test_withIndexPCA.merge(red_df_95, on='time', how='left')

test_withIndexPCA

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,0,0.0,20.0,2016-04-01,0.000004,81.055560,-8.377182,5.273445,2.527817,2.160847,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
1,1,0.0,22.5,2016-04-01,-0.000002,76.221700,-6.143539,4.284462,4.278809,2.030232,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
2,2,0.0,25.0,2016-04-01,0.000018,75.465416,-1.724731,3.315552,3.811935,1.344738,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
3,3,0.0,27.5,2016-04-01,-0.000009,72.850200,1.577347,1.549385,4.243774,2.951813,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
4,4,0.0,30.0,2016-04-01,-0.000009,82.229460,1.983276,1.535709,5.736717,0.056183,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929677,929677,-30.0,80.0,2022-12-31,0.000008,14.955483,13.163338,-4.289364,4.153252,-2.868565,...,-8.209614,1.615797,-20.777699,-6.470835,0.233916,-13.13443,57.994613,26.484964,-8.209614,1.615797
929678,929678,-30.0,82.5,2022-12-31,0.000023,10.061256,13.957420,-4.323608,7.508942,-0.484577,...,-8.209614,1.615797,-20.777699,-6.470835,0.233916,-13.13443,57.994613,26.484964,-8.209614,1.615797
929679,929679,-30.0,85.0,2022-12-31,0.000009,8.901871,15.958916,-3.855770,11.119324,2.067028,...,-8.209614,1.615797,-20.777699,-6.470835,0.233916,-13.13443,57.994613,26.484964,-8.209614,1.615797
929680,929680,-30.0,87.5,2022-12-31,-0.000002,8.630913,19.073456,-3.618012,14.113441,2.629562,...,-8.209614,1.615797,-20.777699,-6.470835,0.233916,-13.13443,57.994613,26.484964,-8.209614,1.615797


In [36]:
test_withIndexPCA.to_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv',index=False)


### latlon*pc95

In [3]:
train = pd.read_csv('/home/b/b382633/TC_adds/indices/train_dailymeans_Sindian.csv')
train

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp95_1,comp95_2,comp95_3,comp95_4,latlon,comp95_0_latlon,comp95_1_latlon,comp95_2_latlon,comp95_3_latlon,comp95_4_latlon
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,60.811630,-37.711484,-22.522648,-17.040834,38.078866,3191.247579,2315.637865,-1436.010543,-857.636868,-648.895623
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,60.811630,-37.711484,-22.522648,-17.040834,35.794553,2999.807844,2176.725076,-1349.865713,-806.188094,-609.969027
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,60.811630,-37.711484,-22.522648,-17.040834,33.541020,2810.947656,2039.684063,-1264.881639,-755.432565,-571.566946
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,60.811630,-37.711484,-22.522648,-17.040834,31.324910,2625.223795,1904.918836,-1181.308862,-705.519913,-533.802593
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,60.811630,-37.711484,-22.522648,-17.040834,29.154759,2443.351563,1772.948433,-1099.469257,-656.642372,-496.821414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,59.862897,-12.051854,-6.211116,13.872671,29.154759,2719.386984,1745.288367,-351.368903,-181.083603,404.454376
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,59.862897,-12.051854,-6.211116,13.872671,31.324910,2921.806067,1875.199878,-377.523243,-194.562662,434.560162
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,59.862897,-12.051854,-6.211116,13.872671,33.541020,3128.511912,2007.862610,-404.231470,-208.327175,465.303519
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,59.862897,-12.051854,-6.211116,13.872671,35.794553,3338.708408,2142.765624,-431.390720,-222.324131,496.566040


In [14]:
train['latitude']+15

0          15.0
1          15.0
2          15.0
3          15.0
4          15.0
           ... 
4268766   -15.0
4268767   -15.0
4268768   -15.0
4268769   -15.0
4268770   -15.0
Name: latitude, Length: 4268771, dtype: float64

In [15]:
train['longitude']-55

0         -35.0
1         -32.5
2         -30.0
3         -27.5
4         -25.0
           ... 
4268766    25.0
4268767    27.5
4268768    30.0
4268769    32.5
4268770    35.0
Name: longitude, Length: 4268771, dtype: float64

In [8]:
#train['latlon'] = np.tile(range(377),len(train.time.unique()))
train['latlon'] = np.sqrt((train['latitude']+15)**2+(train['longitude']-55)**2)
train

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp95_1,comp95_2,comp95_3,comp95_4,latlon,comp95_0_latlon,comp95_1_latlon,comp95_2_latlon,comp95_3_latlon,comp95_4_latlon
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,60.811630,-37.711484,-22.522648,-17.040834,38.078866,0.000000,0.000000,-0.000000,-0.000000,-0.000000
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,60.811630,-37.711484,-22.522648,-17.040834,35.794553,83.806267,60.811630,-37.711484,-22.522648,-17.040834
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,60.811630,-37.711484,-22.522648,-17.040834,33.541020,167.612534,121.623259,-75.422969,-45.045295,-34.081668
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,60.811630,-37.711484,-22.522648,-17.040834,31.324910,251.418802,182.434889,-113.134453,-67.567943,-51.122502
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,60.811630,-37.711484,-22.522648,-17.040834,29.154759,335.225069,243.246518,-150.845938,-90.090590,-68.163336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,59.862897,-12.051854,-6.211116,13.872671,29.154759,34698.003902,22268.997734,-4483.289666,-2310.535279,5160.633479
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,59.862897,-12.051854,-6.211116,13.872671,31.324910,34791.278106,22328.860631,-4495.341520,-2316.746395,5174.506150
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,59.862897,-12.051854,-6.211116,13.872671,33.541020,34884.552310,22388.723528,-4507.393373,-2322.957512,5188.378820
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,59.862897,-12.051854,-6.211116,13.872671,35.794553,34977.826514,22448.586425,-4519.445227,-2329.168628,5202.251491


In [17]:
train['lat'] = train['latitude']+15
train['lon'] = train['longitude']-55


In [9]:
train['comp95_0_latlon'] = train['comp95_0']*train['latlon']
train['comp95_1_latlon'] = train['comp95_1']*train['latlon']
train['comp95_2_latlon'] = train['comp95_2']*train['latlon']
train['comp95_3_latlon'] = train['comp95_3']*train['latlon']
train['comp95_4_latlon'] = train['comp95_4']*train['latlon']


In [18]:
train['comp95_0_lat'] = train['comp95_0']*train['lat']
train['comp95_1_lat'] = train['comp95_1']*train['lat']
train['comp95_2_lat'] = train['comp95_2']*train['lat']
train['comp95_3_lat'] = train['comp95_3']*train['lat']
train['comp95_4_lat'] = train['comp95_4']*train['lat']

train['comp95_0_lon'] = train['comp95_0']*train['lon']
train['comp95_1_lon'] = train['comp95_1']*train['lon']
train['comp95_2_lon'] = train['comp95_2']*train['lon']
train['comp95_3_lon'] = train['comp95_3']*train['lon']
train['comp95_4_lon'] = train['comp95_4']*train['lon']


In [19]:
train

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp95_0_lat,comp95_1_lat,comp95_2_lat,comp95_3_lat,comp95_4_lat,comp95_0_lon,comp95_1_lon,comp95_2_lon,comp95_3_lon,comp95_4_lon
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,1257.094008,912.174443,-565.672266,-337.839713,-255.612509,-2933.219352,-2128.407035,1319.901953,788.292665,596.429187
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,1257.094008,912.174443,-565.672266,-337.839713,-255.612509,-2723.703684,-1976.377961,1225.623242,731.986046,553.827102
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,1257.094008,912.174443,-565.672266,-337.839713,-255.612509,-2514.188016,-1824.348887,1131.344531,675.679427,511.225018
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,1257.094008,912.174443,-565.672266,-337.839713,-255.612509,-2304.672348,-1672.319813,1037.065820,619.372808,468.622933
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,1257.094008,912.174443,-565.672266,-337.839713,-255.612509,-2095.156680,-1520.290739,942.787109,563.066189,426.020848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,-1399.113061,-897.943457,180.777809,93.166745,-208.090060,2331.855101,1496.572428,-301.296348,-155.277909,346.816766
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,-1399.113061,-897.943457,180.777809,93.166745,-208.090060,2565.040611,1646.229671,-331.425983,-170.805699,381.498443
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,-1399.113061,-897.943457,180.777809,93.166745,-208.090060,2798.226121,1795.886914,-361.555618,-186.333490,416.180119
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,-1399.113061,-897.943457,180.777809,93.166745,-208.090060,3031.411631,1945.544157,-391.685253,-201.861281,450.861796


In [20]:
train.to_csv('/home/b/b382633/TC_adds/indices/train_dailymeans_Sindian.csv',index=False)

In [16]:
val = pd.read_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv')
#val['latlon'] = np.tile(range(377),len(val.time.unique()))
val['latlon'] = np.sqrt((val['latitude']+15)**2+(val['longitude']-55)**2)

val['comp95_0_latlon'] = val['comp95_0']*val['latlon']
val['comp95_1_latlon'] = val['comp95_1']*val['latlon']
val['comp95_2_latlon'] = val['comp95_2']*val['latlon']
val['comp95_3_latlon'] = val['comp95_3']*val['latlon']
val['comp95_4_latlon'] = val['comp95_4']*val['latlon']

val.to_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv',index=False)

In [21]:
val = pd.read_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv')

val['lat'] = val['latitude']+15
val['lon'] = val['longitude']-55

val['comp95_0_lat'] = val['comp95_0']*val['lat']
val['comp95_1_lat'] = val['comp95_1']*val['lat']
val['comp95_2_lat'] = val['comp95_2']*val['lat']
val['comp95_3_lat'] = val['comp95_3']*val['lat']
val['comp95_4_lat'] = val['comp95_4']*val['lat']

val['comp95_0_lon'] = val['comp95_0']*val['lon']
val['comp95_1_lon'] = val['comp95_1']*val['lon']
val['comp95_2_lon'] = val['comp95_2']*val['lon']
val['comp95_3_lon'] = val['comp95_3']*val['lon']
val['comp95_4_lon'] = val['comp95_4']*val['lon']


In [17]:
test = pd.read_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv')
#test['latlon'] = np.tile(range(377),len(test.time.unique()))
test['latlon'] = np.sqrt((test['latitude']+15)**2+(test['longitude']-55)**2)

test['comp95_0_latlon'] = test['comp95_0']*test['latlon']
test['comp95_1_latlon'] = test['comp95_1']*test['latlon']
test['comp95_2_latlon'] = test['comp95_2']*test['latlon']
test['comp95_3_latlon'] = test['comp95_3']*test['latlon']
test['comp95_4_latlon'] = test['comp95_4']*test['latlon']

test.to_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv',index=False)

In [22]:
test = pd.read_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv')

test['lat'] = test['latitude']+15
test['lon'] = test['longitude']-55

test['comp95_0_lat'] = test['comp95_0']*test['lat']
test['comp95_1_lat'] = test['comp95_1']*test['lat']
test['comp95_2_lat'] = test['comp95_2']*test['lat']
test['comp95_3_lat'] = test['comp95_3']*test['lat']
test['comp95_4_lat'] = test['comp95_4']*test['lat']

test['comp95_0_lon'] = test['comp95_0']*test['lon']
test['comp95_1_lon'] = test['comp95_1']*test['lon']
test['comp95_2_lon'] = test['comp95_2']*test['lon']
test['comp95_3_lon'] = test['comp95_3']*test['lon']
test['comp95_4_lon'] = test['comp95_4']*test['lon']


In [33]:
val.to_csv('/home/b/b382633/TC_adds/indices/val_dailymeans_Sindian.csv',index=False)
test.to_csv('/home/b/b382633/TC_adds/indices/test_dailymeans_Sindian.csv',index=False)

In [34]:
v = pd.read_csv('/home/b/b382633/TC_adds/indices/'+'val_dailymeans_'+'Sindian'+'.csv')
v.loc[:,'comp95_0_lat']

0         1280.306808
1         1280.306808
2         1280.306808
3         1280.306808
4         1280.306808
             ...     
688397     146.810017
688398     146.810017
688399     146.810017
688400     146.810017
688401     146.810017
Name: comp95_0_lat, Length: 688402, dtype: float64

### checks

In [2]:
import pandas as pd
pd.read_csv("/home/b/b382633/TC_adds/indices/train_dailymeans_Sindian.csv")

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.811630,-37.711484,-22.522648,-17.040834
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.811630,-37.711484,-22.522648,-17.040834
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.811630,-37.711484,-22.522648,-17.040834
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.811630,-37.711484,-22.522648,-17.040834
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,-22.522648,-17.040834,-4.716553,-5.327079,4.680688,83.806267,60.811630,-37.711484,-22.522648,-17.040834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,-6.211116,13.872671,-17.330319,-2.108588,-5.766673,93.274204,59.862897,-12.051854,-6.211116,13.872671


In [12]:
pd.read_csv("/home/b/b382633/TC_adds/predictions/Unet/CNNIndices_Sindian.csv")

,Unnamed: 0.1,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,...,predictions_lag4,predictions_lag5,predictions_lag6,predictions_lag7,predictions_lag8,predictions_lag9,predictions_lag10,predictions_lag11,predictions_lag12,predictions_lag13
0,0,0,0.0,20.0,2016-04-01,0.238973,1.386669,-0.945404,1.574206,0.511504,...,0.000128,0.003040,5.452696e-05,1.447201e-04,1.000986e-04,1.576259e-05,7.880749e-05,0.000358,3.029241e-05,0.000546
1,1,1,0.0,22.5,2016-04-01,-0.172396,1.196522,-0.819827,1.371793,0.717349,...,0.000031,0.000206,9.012886e-09,8.691120e-06,2.640203e-07,1.641364e-07,2.740722e-06,0.000061,8.376418e-06,0.000032
2,2,2,0.0,25.0,2016-04-01,1.188889,1.166772,-0.571398,1.173488,0.662464,...,0.000015,0.000178,5.870336e-09,4.574283e-07,1.354650e-07,2.371623e-08,7.642611e-07,0.000018,1.830778e-06,0.000006
3,3,3,0.0,27.5,2016-04-01,-0.649707,1.063899,-0.385753,0.812011,0.713230,...,0.000008,0.000077,8.689971e-09,2.209202e-07,5.619184e-08,1.555464e-08,6.022574e-08,0.000017,9.638123e-07,0.000004
4,4,4,0.0,30.0,2016-04-01,-0.629183,1.432846,-0.362931,0.809212,0.888739,...,0.000009,0.000072,1.501120e-08,1.939304e-07,7.135770e-08,1.960775e-08,7.092633e-08,0.000021,1.211826e-06,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929677,929677,929677,-30.0,80.0,2022-12-31,0.532980,-1.213473,0.265621,-0.382993,0.702588,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000
929678,929678,929678,-30.0,82.5,2022-12-31,1.476765,-1.405994,0.310265,-0.390002,1.097079,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000
929679,929679,929679,-30.0,85.0,2022-12-31,0.551509,-1.451600,0.422791,-0.294250,1.521511,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000
929680,929680,929680,-30.0,87.5,2022-12-31,-0.176586,-1.462259,0.597893,-0.245589,1.873496,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000


In [13]:
pd.read_csv("/home/b/b382633/TC_adds/predictions/Unet/UnetIndices_Sindian.csv")

,Unnamed: 0.1,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,...,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,0,0,0.0,20.0,2016-04-01,0.238973,1.386669,-0.945404,1.574206,0.511504,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
1,1,1,0.0,22.5,2016-04-01,-0.172396,1.196522,-0.819827,1.371793,0.717349,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
2,2,2,0.0,25.0,2016-04-01,1.188889,1.166772,-0.571398,1.173488,0.662464,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
3,3,3,0.0,27.5,2016-04-01,-0.649707,1.063899,-0.385753,0.812011,0.713230,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
4,4,4,0.0,30.0,2016-04-01,-0.629183,1.432846,-0.362931,0.809212,0.888739,...,3.848911,-11.034493,29.152022,6.941696,3.588352,-18.004323,6.175446,11.884714,3.848911,-11.034493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216379,216379,216379,-30.0,45.0,2017-10-26,1.477462,-0.373441,1.214900,1.903132,0.572525,...,23.253478,19.529633,-5.782219,7.421894,1.774476,-54.361022,9.831171,6.831686,23.253478,19.529633
216380,216380,216380,-30.0,47.5,2017-10-26,-0.506605,-0.194245,1.067558,1.453790,1.123043,...,23.253478,19.529633,-5.782219,7.421894,1.774476,-54.361022,9.831171,6.831686,23.253478,19.529633
216381,216381,216381,-30.0,50.0,2017-10-26,0.376923,-0.486296,0.831773,1.405315,1.505489,...,23.253478,19.529633,-5.782219,7.421894,1.774476,-54.361022,9.831171,6.831686,23.253478,19.529633
216382,216382,216382,-30.0,52.5,2017-10-26,0.869699,-1.437952,0.523767,1.330355,1.466460,...,23.253478,19.529633,-5.782219,7.421894,1.774476,-54.361022,9.831171,6.831686,23.253478,19.529633


### Middle of network

In [4]:
indices_95 = pd.read_csv('/home/b/b382633/TC_adds/indices/red_df_95.csv')
indices_95

,time,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,1980-01-01,83.806267,60.811630,-37.711484,-22.522648,-17.040834
1,1980-01-02,74.552681,62.812590,-34.296830,-22.060617,-26.121479
2,1980-01-03,42.879099,62.979958,-26.535009,-21.568343,-30.888566
3,1980-01-04,39.239566,64.712265,-20.704652,-20.293211,-36.299514
4,1980-01-05,59.075320,63.086554,-23.310044,-18.588885,-23.402192
...,...,...,...,...,...,...
15945,2023-08-28,-2.690358,-53.624069,31.383152,27.083869,-1.433984
15946,2023-08-29,12.192680,-53.413492,21.551420,25.770639,1.572338
15947,2023-08-30,-0.572852,-56.304293,17.369056,26.600480,6.799272
15948,2023-08-31,-0.360282,-57.699650,15.826201,25.263325,3.404456


In [5]:
indices_99 = pd.read_csv('/home/b/b382633/TC_adds/indices/red_df_99.csv')
indices_99

,time,comp99_0,comp99_1,comp99_2,comp99_3,comp99_4,comp99_5,comp99_6,comp99_7
0,1980-01-01,83.806267,60.811630,-37.711484,-22.522648,-17.040834,-4.716553,-5.327079,4.680688
1,1980-01-02,74.552681,62.812590,-34.296830,-22.060617,-26.121479,-3.582355,-4.297971,1.700701
2,1980-01-03,42.879099,62.979958,-26.535009,-21.568343,-30.888566,-3.567491,-1.976986,-1.825118
3,1980-01-04,39.239566,64.712265,-20.704652,-20.293211,-36.299514,-1.978507,1.755396,-2.470499
4,1980-01-05,59.075320,63.086554,-23.310044,-18.588885,-23.402192,-5.655348,6.397826,-3.037748
...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-2.690358,-53.624069,31.383152,27.083869,-1.433984,-10.555020,0.699701,4.365854
15946,2023-08-29,12.192680,-53.413492,21.551420,25.770639,1.572338,-13.197540,-0.713921,4.022209
15947,2023-08-30,-0.572852,-56.304293,17.369056,26.600480,6.799272,-14.412565,-1.223100,2.077897
15948,2023-08-31,-0.360282,-57.699650,15.826201,25.263325,3.404456,-15.439204,0.600153,-0.812687


In [3]:
train_img_std = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiantrain_img_std.npy')
val_img_std = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindianval_img_std.npy')
test_img_std = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiantest_img_std.npy')

y_train_img = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiany_train_img.npy')
y_val_img = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiany_val_img.npy')
y_test_img = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiany_test_img.npy')


#### 1) standard CNN autoencoder directly on target 

In [25]:
### lag = 0

#model = Sequential()
#model.add(layers.Input(shape=(13, 29, train_img_std.shape[3])))
#model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)


autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

#model.add(layers.Flatten())
#model.add(layers.Dense(64))
#model.add(layers.Reshape((2, 4, 8)))
#model.add(layers.UpSampling2D((4,4)))
#
##model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
##model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(9, (3, 3), activation='sigmoid', padding='same')) # sigmoidal activation to have probabilities
#model.add(layers.Cropping2D(cropping=((1,2),(1,2))))
#model.summary()
#

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_73 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 7, 15, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_74 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_75 (Conv2D)          (None, 4, 8, 8)           584 

In [49]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder.fit(train_img_std, y_train_img, validation_data=(val_img_std,y_val_img),
            callbacks=[monitor],epochs=200)
t = autoencoder.predict(test_img_std)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Epoch 1/200
354/354 [==============================] - 3s 8ms/step - loss: 0.0094 - val_loss: 0.0106
Epoch 2/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0092 - val_loss: 0.0090
Epoch 3/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0090 - val_loss: 0.0094
Epoch 4/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0090 - val_loss: 0.0092
Epoch 5/200
354/354 [==============================] - 2s 7ms/step - loss: 0.0090 - val_loss: 0.0097
Epoch 6/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0088 - val_loss: 0.0097
Epoch 7/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0088 - val_loss: 0.0090
Epoch 8/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0086 - val_loss: 0.0093
Epoch 9/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0086 - val_loss: 0.0093
Epoch 10/200
354/354 [==============================] - 3s 7ms/step - loss: 0.0086 - val_lo

In [53]:
### lag = 0

#model = Sequential()
#model.add(layers.Input(shape=(13, 29, train_img_std.shape[3])))
#model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(64)(encoded)

x = layers.Reshape((2, 4, 8))(encoded)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)


autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder.fit(train_img_std, y_train_img, validation_data=(val_img_std,y_val_img),
            callbacks=[monitor],epochs=200)
t = autoencoder.predict(test_img_std)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_95 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 7, 15, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_96 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_97 (Conv2D)          (None, 4, 8, 8)           584

#### 2) simpler

In [58]:
### lag = 0

#model = Sequential()
#model.add(layers.Input(shape=(13, 29, train_img_std.shape[3])))
#model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2, 2), padding='same')(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
#x = layers.UpSampling2D((2, 2))(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)


autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

#model.add(layers.Flatten())
#model.add(layers.Dense(64))
#model.add(layers.Reshape((2, 4, 8)))
#model.add(layers.UpSampling2D((4,4)))
#
##model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
##model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(9, (3, 3), activation='sigmoid', padding='same')) # sigmoidal activation to have probabilities
#model.add(layers.Cropping2D(cropping=((1,2),(1,2))))
#model.summary()
#

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder.fit(train_img_std, y_train_img, validation_data=(val_img_std,y_val_img),
            callbacks=[monitor],epochs=200)
t = autoencoder.predict(test_img_std)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_119 (Conv2D)         (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_52 (MaxPoolin  (None, 7, 15, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_120 (Conv2D)         (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_53 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_121 (Conv2D)         (None, 4, 8, 8)           584

In [59]:
### lag = 0

#model = Sequential()
#model.add(layers.Input(shape=(13, 29, train_img_std.shape[3])))
#model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2, 2), padding='same')(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(256)(encoded)

x = layers.Reshape((4, 8, 8))(encoded)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.UpSampling2D((2, 2))(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)


autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

#model.add(layers.Flatten())
#model.add(layers.Dense(64))
#model.add(layers.Reshape((2, 4, 8)))
#model.add(layers.UpSampling2D((4,4)))
#
##model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
##model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(9, (3, 3), activation='sigmoid', padding='same')) # sigmoidal activation to have probabilities
#model.add(layers.Cropping2D(cropping=((1,2),(1,2))))
#model.summary()
#

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder.fit(train_img_std, y_train_img, validation_data=(val_img_std,y_val_img),
            callbacks=[monitor],epochs=200)
t = autoencoder.predict(test_img_std)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_124 (Conv2D)         (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_54 (MaxPoolin  (None, 7, 15, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_125 (Conv2D)         (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 256)               0  

#### 3) Autoencoder for features with and without flattening, autoencoders of section 1 and 2

In [8]:
### no flattening

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(9, (3, 3), activation='linear', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)


autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder.fit(train_img_std, train_img_std, validation_data=(val_img_std,val_img_std),
            callbacks=[monitor],epochs=2000)
t = autoencoder.predict(test_img_std)


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 15, 16)        0         
 2D)                                                             
                                                                 
 conv2d_22 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 4, 8, 8)           584 

In [9]:
autoencoder.save('featuresAutoencoderComplexNoflattening_model.keras')

In [6]:
### flattening

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(64)(encoded)

x = layers.Reshape((2, 4, 8))(encoded)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(9, (3, 3), activation='linear', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)


autoencoder1 = keras.Model(input_img, decoded)
autoencoder1.compile(optimizer='adam', loss='mse')
autoencoder1.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder1.fit(train_img_std, train_img_std, validation_data=(val_img_std,val_img_std),
            callbacks=[monitor],epochs=2000)
t = autoencoder1.predict(test_img_std)


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 15, 16)        0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 8, 8)          0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 4, 8, 8)           584 

In [7]:
autoencoder1.save('featuresAutoencoderComplexFlattening_model.keras')

In [8]:
### no flattening, simpler
input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2, 2), padding='same')(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
#x = layers.UpSampling2D((2, 2))(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(9, (3, 3), activation='linear', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)

autoencoder2 = keras.Model(input_img, decoded)
autoencoder2.compile(optimizer='adam', loss='mse')
autoencoder2.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder2.fit(train_img_std, train_img_std, validation_data=(val_img_std,val_img_std),
            callbacks=[monitor],epochs=2000)
t = autoencoder2.predict(test_img_std)


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_17 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 15, 16)        0         
 2D)                                                             
                                                                 
 conv2d_18 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 8, 8)          0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 4, 8, 8)           584 

In [9]:
autoencoder2.save('featuresAutoencoderSimpleNoflattening_model.keras')

In [11]:
### simpler, flattening

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2, 2), padding='same')(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(256)(encoded)

x = layers.Reshape((4, 8, 8))(encoded)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.UpSampling2D((2, 2))(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(9, (3, 3), activation='linear', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)

autoencoder3 = keras.Model(input_img, decoded)
autoencoder3.compile(optimizer='adam', loss='mse')
autoencoder3.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder3.fit(train_img_std, train_img_std, validation_data=(val_img_std,val_img_std),
            callbacks=[monitor],epochs=2000)
t = autoencoder3.predict(test_img_std)


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 15, 16)        0         
 2D)                                                             
                                                                 
 conv2d_23 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 256)               0   

In [12]:
autoencoder3.save('featuresAutoencoderSimpleFlattening_model.keras')

In [13]:
### simpler, flattening

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2, 2), padding='same')(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(64)(encoded)

x = layers.Reshape((2, 4, 8))(encoded)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(9, (3, 3), activation='linear', padding='same')(x)
decoded = layers.Cropping2D(cropping=((1,2),(1,2)))(decoded)

autoencoder4 = keras.Model(input_img, decoded)
autoencoder4.compile(optimizer='adam', loss='mse')
autoencoder4.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder4.fit(train_img_std, train_img_std, validation_data=(val_img_std,val_img_std),
            callbacks=[monitor],epochs=2000)
t = autoencoder4.predict(test_img_std)


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 13, 29, 9)]       0         
                                                                 
 conv2d_27 (Conv2D)          (None, 13, 29, 16)        1312      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 7, 15, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 7, 15, 8)          1160      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 256)               0   

In [19]:
### all flat

#input_img = keras.Input((13, 29, train_img_std.shape[3]))
#
#x = layers.Flatten()(input_img)
#x = layers.Dense(32, activation='relu')(x)
##x = layers.Dense(32, activation='relu')(x)
#encoded = layers.Dense(16, activation='relu')(x)
#
#x = layers.Dense(32, activation='relu')(encoded)
##x = layers.Dense(64, activation='relu')(x)
#decoded = layers.Dense(3393, activation='relu')(x)
#decoded = layers.Reshape((13, 29, 9))(decoded)
#
#autoencoder5 = keras.Model(input_img, decoded)
#autoencoder5.compile(optimizer='adam', loss='mse')
#autoencoder5.summary()
#
#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
#                            verbose=1, mode='auto', restore_best_weights=True)
#
#autoencoder5.fit(train_img_std, train_img_std, validation_data=(val_img_std,val_img_std),
#            callbacks=[monitor],epochs=2000)
#t = autoencoder5.predict(test_img_std)
#

### 4) Load the four autoencoders, learn target

In [7]:
autoencoder_complex_noFlattening = tf.keras.models.load_model('featuresAutoencoderComplexNoflattening_model.keras')
autoencoder_simple_noFlattening = tf.keras.models.load_model('featuresAutoencoderSimpleNoflattening_model.keras')

autoencoder_complex_flattening = tf.keras.models.load_model('featuresAutoencoderComplexFlattening_model.keras')
autoencoder_simple_flattening = tf.keras.models.load_model('featuresAutoencoderSimpleFlattening_model.keras')


#### 4.1.1

In [25]:
from sklearn.metrics import mean_squared_error
mean_squared_error(val_img_std.reshape(-1,1),autoencoder_complex_noFlattening.predict(val_img_std).reshape(-1,1))

58/58 [==============================] - 0s 2ms/step


0.23452861503933561

In [42]:
intermediate_model = Model(inputs=autoencoder_complex_noFlattening.layers[0].input, 
                              outputs=autoencoder_complex_noFlattening.layers[6].output)
train_img_std_encoded = intermediate_model.predict(train_img_std)
val_img_std_encoded = intermediate_model.predict(val_img_std)
test_img_std_encoded = intermediate_model.predict(test_img_std)

78/78 [==============================] - 0s 1ms/step


In [48]:
### lag = 0

input_img = keras.Input((train_img_std_encoded.shape[1:]))

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(train_img_std_encoded, y_train_img, validation_data=(val_img_std_encoded,y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(test_img_std_encoded)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 2, 4, 8)]         0         
                                                                 
 conv2d_45 (Conv2D)          (None, 2, 4, 8)           584       
                                                                 
 up_sampling2d_23 (UpSamplin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_46 (Conv2D)          (None, 4, 8, 8)           584       
                                                                 
 up_sampling2d_24 (UpSamplin  (None, 8, 16, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 8, 16, 16)         116

#### 4.1.2

In [8]:
from sklearn.metrics import mean_squared_error
mean_squared_error(val_img_std.reshape(-1,1),autoencoder_complex_flattening.predict(val_img_std).reshape(-1,1))

58/58 [==============================] - 0s 2ms/step


0.2359178886159138

In [9]:
intermediate_model = Model(inputs=autoencoder_complex_flattening.layers[0].input, 
                              outputs=autoencoder_complex_flattening.layers[8].output)
train_img_std_encoded = intermediate_model.predict(train_img_std)
val_img_std_encoded = intermediate_model.predict(val_img_std)
test_img_std_encoded = intermediate_model.predict(test_img_std)

78/78 [==============================] - 0s 1ms/step


In [57]:
### lag = 0

input_img = keras.Input((train_img_std_encoded.shape[1:]))

x = layers.Reshape((2, 4, 8))(input_img)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(train_img_std_encoded, y_train_img, validation_data=(val_img_std_encoded,y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(test_img_std_encoded)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 64)]              0         
                                                                 
 reshape_8 (Reshape)         (None, 2, 4, 8)           0         
                                                                 
 conv2d_49 (Conv2D)          (None, 2, 4, 8)           584       
                                                                 
 up_sampling2d_26 (UpSamplin  (None, 4, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_50 (Conv2D)          (None, 4, 8, 8)           584       
                                                                 
 up_sampling2d_27 (UpSamplin  (None, 8, 16, 8)         0         
 g2D)                                                     

In [13]:
### with 95% PCA indices

### lag = 0

input_img = keras.Input((69))

x = layers.Dense((64))(input_img)

x = layers.Reshape((2, 4, 8))(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(np.concatenate((train_img_std_encoded,indices_95.iloc[0:11323,1:].values),axis=1), y_train_img, validation_data=(np.concatenate((val_img_std_encoded,indices_95.iloc[11323:13149,1:].values),axis=1),y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(np.concatenate((test_img_std_encoded,indices_95.iloc[13240:-244,1:]),axis=1))

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 69)]              0         
                                                                 
 dense (Dense)               (None, 64)                4480      
                                                                 
 reshape (Reshape)           (None, 2, 4, 8)           0         
                                                                 
 conv2d (Conv2D)             (None, 2, 4, 8)           584       
                                                                 
 up_sampling2d (UpSampling2D  (None, 4, 8, 8)          0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 8, 8)           584       
                                                           

In [14]:
### with 99% PCA indices

### lag = 0

input_img = keras.Input((72))

x = layers.Dense((64))(input_img)

x = layers.Reshape((2, 4, 8))(x)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(np.concatenate((train_img_std_encoded,indices_99.iloc[0:11323,1:].values),axis=1), y_train_img, validation_data=(np.concatenate((val_img_std_encoded,indices_99.iloc[11323:13149,1:].values),axis=1),y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(np.concatenate((test_img_std_encoded,indices_99.iloc[13240:-244,1:]),axis=1))

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 72)]              0         
                                                                 
 dense_1 (Dense)             (None, 64)                4672      
                                                                 
 reshape_1 (Reshape)         (None, 2, 4, 8)           0         
                                                                 
 conv2d_4 (Conv2D)           (None, 2, 4, 8)           584       
                                                                 
 up_sampling2d_3 (UpSampling  (None, 4, 8, 8)          0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 8, 8)           584       
                                                           

In [15]:
### with 99% PCA indices

### lag = 0

input_img = keras.Input((72))

#x = layers.Dense((64))(input_img)

x = layers.Reshape((2, 4, 9))(input_img)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(np.concatenate((train_img_std_encoded,indices_99.iloc[0:11323,1:].values),axis=1), y_train_img, validation_data=(np.concatenate((val_img_std_encoded,indices_99.iloc[11323:13149,1:].values),axis=1),y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(np.concatenate((test_img_std_encoded,indices_99.iloc[13240:-244,1:]),axis=1))

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 72)]              0         
                                                                 
 reshape_2 (Reshape)         (None, 2, 4, 9)           0         
                                                                 
 conv2d_8 (Conv2D)           (None, 2, 4, 8)           656       
                                                                 
 up_sampling2d_6 (UpSampling  (None, 4, 8, 8)          0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 4, 8, 8)           584       
                                                                 
 up_sampling2d_7 (UpSampling  (None, 8, 16, 8)         0         
 2D)                                                       

#### 4.2.1

In [58]:
from sklearn.metrics import mean_squared_error
mean_squared_error(val_img_std.reshape(-1,1),autoencoder_simple_noFlattening.predict(val_img_std).reshape(-1,1))

58/58 [==============================] - 0s 2ms/step


0.11622056718955961

In [64]:
intermediate_model = Model(inputs=autoencoder_simple_noFlattening.layers[0].input, 
                              outputs=autoencoder_simple_noFlattening.layers[4].output)
train_img_std_encoded = intermediate_model.predict(train_img_std)
val_img_std_encoded = intermediate_model.predict(val_img_std)
test_img_std_encoded = intermediate_model.predict(test_img_std)

78/78 [==============================] - 0s 981us/step


In [66]:
### lag = 0

input_img = keras.Input((train_img_std_encoded.shape[1:]))

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(train_img_std_encoded, y_train_img, validation_data=(val_img_std_encoded,y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(test_img_std_encoded)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 4, 8, 8)]         0         
                                                                 
 conv2d_53 (Conv2D)          (None, 4, 8, 8)           584       
                                                                 
 up_sampling2d_29 (UpSamplin  (None, 8, 16, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_54 (Conv2D)          (None, 8, 16, 16)         1168      
                                                                 
 up_sampling2d_30 (UpSamplin  (None, 16, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_55 (Conv2D)          (None, 16, 32, 1)         145

#### 4.2.2

In [67]:
from sklearn.metrics import mean_squared_error
mean_squared_error(val_img_std.reshape(-1,1),autoencoder_simple_flattening.predict(val_img_std).reshape(-1,1))

58/58 [==============================] - 0s 2ms/step


0.1322124076391697

In [70]:
intermediate_model = Model(inputs=autoencoder_simple_flattening.layers[0].input, 
                              outputs=autoencoder_simple_flattening.layers[6].output)
train_img_std_encoded = intermediate_model.predict(train_img_std)
val_img_std_encoded = intermediate_model.predict(val_img_std)
test_img_std_encoded = intermediate_model.predict(test_img_std)

78/78 [==============================] - 0s 1ms/step


In [73]:
### lag = 0

input_img = keras.Input((train_img_std_encoded.shape[1:]))

x = layers.Reshape((4, 8, 8))(input_img)

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
output = layers.Cropping2D(cropping=((1,2),(1,2)))(output)


final_model = keras.Model(input_img, output)
final_model.compile(optimizer='adam', loss='binary_crossentropy')
final_model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

final_model.fit(train_img_std_encoded, y_train_img, validation_data=(val_img_std_encoded,y_val_img),
            callbacks=[monitor],epochs=2000)
t = final_model.predict(test_img_std_encoded)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 256)]             0         
                                                                 
 reshape_10 (Reshape)        (None, 4, 8, 8)           0         
                                                                 
 conv2d_56 (Conv2D)          (None, 4, 8, 8)           584       
                                                                 
 up_sampling2d_31 (UpSamplin  (None, 8, 16, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 8, 16, 16)         1168      
                                                                 
 up_sampling2d_32 (UpSamplin  (None, 16, 32, 16)       0         
 g2D)                                                     

In [72]:
train_img_std_encoded.shape

(11323, 256)

# CNN of 2) with flattening, then addition of index

In [26]:
from tensorflow.keras.layers.experimental.preprocessing import Resizing


In [38]:
### lag = 0

#model = Sequential()
#model.add(layers.Input(shape=(13, 29, train_img_std.shape[3])))
#model.add(layers.Conv2D(8, (3,3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.MaxPooling2D((2, 2), padding='same'))

input_img = keras.Input((13, 29, train_img_std.shape[3]))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2, 2), padding='same')(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Flatten()(encoded)

inputs_2 = Input(shape = (8))
concat = concatenate([encoded,inputs_2])

#concat = layers.Dense(256)(concat)

x = layers.Reshape((3, 8, 11))(concat)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#x = layers.UpSampling2D((2, 2))(x)
#x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((3, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
decoded = layers.Cropping2D(cropping=((2,3),(1,2)))(decoded)

autoencoder = keras.Model([input_img,inputs_2], decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

#model.add(layers.Flatten())
#model.add(layers.Dense(64))
#model.add(layers.Reshape((2, 4, 8)))
#model.add(layers.UpSampling2D((4,4)))
#
##model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
##model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
#model.add(layers.UpSampling2D((2,2)))
#model.add(layers.Conv2D(9, (3, 3), activation='sigmoid', padding='same')) # sigmoidal activation to have probabilities
#model.add(layers.Cropping2D(cropping=((1,2),(1,2))))
#model.summary()
#

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

autoencoder.fit([train_img_std,indices_99.iloc[0:11323,1:]], y_train_img, validation_data=([val_img_std,indices_99.iloc[11323:13149,1:]],y_val_img),
            callbacks=[monitor],epochs=200)
t = autoencoder.predict([test_img_std,indices_99.iloc[13240:-244,1:]])

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))


Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_42 (InputLayer)          [(None, 13, 29, 9)]  0           []                               
                                                                                                  
 conv2d_83 (Conv2D)             (None, 13, 29, 16)   1312        ['input_42[0][0]']               
                                                                                                  
 max_pooling2d_38 (MaxPooling2D  (None, 7, 15, 16)   0           ['conv2d_83[0][0]']              
 )                                                                                                
                                                                                                  
 conv2d_84 (Conv2D)             (None, 7, 15, 8)     1160        ['max_pooling2d_38[0][0]']

# Unet with flattening and addition of indices in the middle 

In [7]:
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    input_img = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = input_img

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    flattened = layers.Flatten()(c5)
    
    inputs_2 = Input(shape = (8))
    concat = concatenate([flattened,inputs_2])

    dense = layers.Dense(512)(concat)
    dense = layers.Reshape((1, 2, 256))(dense)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(dense)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    c10 = Cropping2D(cropping=((1,2),(1,2)))(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c10)
     
    model = Model(inputs=[input_img,inputs_2], outputs=[outputs])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [8]:
model = simple_unet_model(16,32,9)
#model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 32, 9)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 32, 16)   1312        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 16, 32, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 32, 16)   2320        ['dropout[0][0]']                
                                                                                              

In [7]:
train_img_std = np.pad(train_img_std, ((0,0),(1,2),(1,2),(0,0)), 'constant')
val_img_std = np.pad(val_img_std, ((0,0),(1,2),(1,2),(0,0)), 'constant')
test_img_std = np.pad(test_img_std, ((0,0),(1,2),(1,2),(0,0)), 'constant')


In [10]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

model.fit([train_img_std,indices_99.iloc[0:11323,1:]], y_train_img, validation_data=([val_img_std,indices_99.iloc[11323:13149,1:]],y_val_img),
            callbacks=[monitor],epochs=200)
t = model.predict([test_img_std,indices_99.iloc[13240:-244,1:]])

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))


Epoch 1/200
354/354 [==============================] - 15s 36ms/step - loss: 0.0181 - accuracy: 0.9960 - val_loss: 0.0082 - val_accuracy: 0.9971
Epoch 2/200
354/354 [==============================] - 12s 35ms/step - loss: 0.0106 - accuracy: 0.9966 - val_loss: 0.0088 - val_accuracy: 0.9968
Epoch 3/200
354/354 [==============================] - 12s 35ms/step - loss: 0.0098 - accuracy: 0.9968 - val_loss: 0.0108 - val_accuracy: 0.9964
Epoch 4/200
354/354 [==============================] - 12s 35ms/step - loss: 0.0094 - accuracy: 0.9968 - val_loss: 0.0078 - val_accuracy: 0.9972
Epoch 5/200
354/354 [==============================] - 12s 35ms/step - loss: 0.0091 - accuracy: 0.9969 - val_loss: 0.0083 - val_accuracy: 0.9971
Epoch 6/200
354/354 [==============================] - 13s 35ms/step - loss: 0.0088 - accuracy: 0.9970 - val_loss: 0.0075 - val_accuracy: 0.9973
Epoch 7/200
354/354 [==============================] - 12s 35ms/step - loss: 0.0085 - accuracy: 0.9970 - val_loss: 0.0076 - val_ac

## Check

In [8]:
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    input_img = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = input_img

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    flattened = layers.Flatten()(c5)
    
    #inputs_2 = Input(shape = (8))
    #concat = concatenate([flattened,inputs_2])

    dense = layers.Dense(512)(flattened)
    dense = layers.Reshape((1, 2, 256))(dense)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(dense)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    c10 = Cropping2D(cropping=((1,2),(1,2)))(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c10)
     
    model = Model(inputs=[input_img], outputs=[outputs])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
model = simple_unet_model(16,32,9)

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=50, 
                            verbose=1, mode='auto', restore_best_weights=True)

model.fit(train_img_std, y_train_img, validation_data=(val_img_std,y_val_img),
            callbacks=[monitor],epochs=200)
t = model.predict(test_img_std)

print(f'All zeros Brier score: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}') 

df_target = pd.read_csv("/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv")
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - t.reshape(-1,1)) ** 2))
print(np.average((df_target.new_target.values.reshape(-1,1)/100 - np.zeros(df_target.shape[0]).reshape(-1,1)) ** 2))


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 32, 9)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 32, 16)   1312        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 16, 32, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 32, 16)   2320        ['dropout[0][0]']                
                                                                                              

# Usual Unet with all multiplication with components as additional channels

In [3]:
indices_95 = pd.read_csv('/home/b/b382633/TC_adds/indices/red_df_95.csv')
indices_95

,time,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4
0,1980-01-01,83.806267,60.811630,-37.711484,-22.522648,-17.040834
1,1980-01-02,74.552681,62.812590,-34.296830,-22.060617,-26.121479
2,1980-01-03,42.879099,62.979958,-26.535009,-21.568343,-30.888566
3,1980-01-04,39.239566,64.712265,-20.704652,-20.293211,-36.299514
4,1980-01-05,59.075320,63.086554,-23.310044,-18.588885,-23.402192
...,...,...,...,...,...,...
15945,2023-08-28,-2.690358,-53.624069,31.383152,27.083869,-1.433984
15946,2023-08-29,12.192680,-53.413492,21.551420,25.770639,1.572338
15947,2023-08-30,-0.572852,-56.304293,17.369056,26.600480,6.799272
15948,2023-08-31,-0.360282,-57.699650,15.826201,25.263325,3.404456


In [15]:
indices_95_std=(indices_95.iloc[:,1:]-indices_95.iloc[:,1:].mean())/indices_95.iloc[:,1:].std()
indices_95_std['time'] = indices_95['time']
indices_95_std

,comp95_0,comp95_1,comp95_2,comp95_3,comp95_4,time
0,1.412493,1.636958,-2.009995,-1.221545,-1.146112,1980-01-01
1,1.256530,1.690821,-1.827997,-1.196486,-1.756848,1980-01-02
2,0.722695,1.695326,-1.414297,-1.169787,-2.077467,1980-01-03
3,0.661354,1.741957,-1.103543,-1.100629,-2.441390,1980-01-04
4,0.995671,1.698195,-1.242409,-1.008192,-1.573957,1980-01-05
...,...,...,...,...,...,...
15945,-0.045344,-1.443479,1.672700,1.468929,-0.096445,2023-08-28
15946,0.205499,-1.437811,1.148675,1.397704,0.105750,2023-08-29
15947,-0.009655,-1.515627,0.925758,1.442711,0.457297,2023-08-30
15948,-0.006072,-1.553188,0.843525,1.370189,0.228973,2023-08-31


In [5]:
indices_99 = pd.read_csv('/home/b/b382633/TC_adds/indices/red_df_99.csv')


In [22]:
train_img_std = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiantrain_img_std.npy')
val_img_std = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindianval_img_std.npy')
test_img_std = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiantest_img_std.npy')

y_train_img = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiany_train_img.npy')
y_val_img = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiany_val_img.npy')
y_test_img = np.load('/home/b/b382633/TC_adds/csv_extracted/Sindiany_test_img.npy')


In [4]:
y_train_img_noBinary = np.load('/home/b/b382633/TC_adds/predictions/Unet/Sindiany_train_img_noBinary.npy')
y_train_img_noBinary


array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [8]:
from tensorflow.keras.losses import BinaryCrossentropy
loss = BinaryCrossentropy()
loss(y_train_img_noBinary,y_train_img).numpy()

-4.232977096537362

In [27]:
### cycle over all days and multiply the constant value of each feature by the value of the PC0 of the index

train_img_std_full = train_img_std

for comp in range(5):
    aux_img_std = np.zeros(list(train_img_std.shape))
    for day in range(train_img_std.shape[0]):
        print(day)
        aux_img_std[day,:,:,:] = train_img_std[day,:,:,:]*indices_95_std.loc[indices_95_std.time<='2010-12-31'].iloc[day,comp]
    train_img_std_full = np.concatenate((train_img_std_full,aux_img_std),axis=3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
val_img_std_full = val_img_std

for comp in range(5):
    aux_img_std = np.zeros(list(val_img_std.shape))
    for day in range(val_img_std.shape[0]):
        print(day)
        aux_img_std[day,:,:,:] = val_img_std[day,:,:,:]*indices_95_std.loc[(indices_95_std.time>'2010-12-31') & (indices_95_std.time<='2015-12-31')].iloc[day,comp]
    val_img_std_full = np.concatenate((val_img_std_full,aux_img_std),axis=3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [40]:
test_img_std_full = test_img_std

for comp in range(5):
    aux_img_std = np.zeros(list(test_img_std.shape))
    for day in range(test_img_std.shape[0]):
        print(day)
        aux_img_std[day,:,:,:] = test_img_std[day,:,:,:]*indices_95_std.loc[(indices_95_std.time>='2016-04-01') & (indices_95_std.time<='2022-12-31')].iloc[day,comp]
    test_img_std_full = np.concatenate((test_img_std_full,aux_img_std),axis=3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [41]:
print(train_img_std.shape,val_img_std.shape,test_img_std.shape)

(11323, 13, 29, 9) (1826, 13, 29, 9) (2466, 13, 29, 9)


In [42]:
print(train_img_std_full.shape,val_img_std_full.shape,test_img_std_full.shape)

(11323, 13, 29, 54) (1826, 13, 29, 54) (2466, 13, 29, 54)


In [45]:
np.save('/home/b/b382633/TC_adds/csv_extracted/Sindiantrain_img_std_indices.npy',train_img_std_full)
np.save('/home/b/b382633/TC_adds/csv_extracted/Sindianval_img_std_indices.npy',val_img_std_full)
np.save('/home/b/b382633/TC_adds/csv_extracted/Sindiantest_img_std_indices.npy',test_img_std_full)


In [17]:
pd.read_csv('/home/b/b382633/TC_adds/csv_extracted/target_train_dailymeans_Sindian.csv')

,Unnamed: 0,latitude,longitude,time,p131089,new_target
0,0,0.0,20.0,1980-01-01,0.0,0.0
1,1,0.0,22.5,1980-01-01,0.0,0.0
2,2,0.0,25.0,1980-01-01,0.0,0.0
3,3,0.0,27.5,1980-01-01,0.0,0.0
4,4,0.0,30.0,1980-01-01,0.0,0.0
...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.0,0.0
4268767,4268767,-30.0,82.5,2010-12-31,0.0,0.0
4268768,4268768,-30.0,85.0,2010-12-31,0.0,0.0
4268769,4268769,-30.0,87.5,2010-12-31,0.0,0.0


In [26]:
def custom_brier(y_true, y_pred):
    y_true = y_true.reshape(-1,)
    y_pred = y_pred.reshape(-1,)
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


In [8]:
y_train_img.shape

(11323, 13, 29, 1)

In [29]:
custom_brier(y_train_img, y_train_img_noBinary/100).numpy()

0.00044142603925111

In [30]:
from tensorflow.keras.losses import MeanSquaredError

loss = MeanSquaredError()
loss(y_train_img_noBinary/100,y_train_img).numpy()

0.00044142603925111

In [2]:
import pandas as pd
pd.read_csv('/home/b/b382633/TC_adds/csv_extracted/target_test_dailymeans_Sindian.csv')

,Unnamed: 0,latitude,longitude,time,p131089,new_target
0,0,0.0,20.0,2016-04-01,0.0,0.0
1,1,0.0,22.5,2016-04-01,0.0,0.0
2,2,0.0,25.0,2016-04-01,0.0,0.0
3,3,0.0,27.5,2016-04-01,0.0,0.0
4,4,0.0,30.0,2016-04-01,0.0,0.0
...,...,...,...,...,...,...
929677,929677,-30.0,80.0,2022-12-31,0.0,0.0
929678,929678,-30.0,82.5,2022-12-31,0.0,0.0
929679,929679,-30.0,85.0,2022-12-31,0.0,0.0
929680,929680,-30.0,87.5,2022-12-31,0.0,0.0


## Unet with all 54 combinations original features + 5 components

In [46]:
train_img_std_padded = np.pad(train_img_std_full, ((0,0),(1,2),(1,2),(0,0)), 'constant')
val_img_std_padded = np.pad(val_img_std_full, ((0,0),(1,2),(1,2),(0,0)), 'constant')
test_img_std_padded = np.pad(test_img_std_full, ((0,0),(1,2),(1,2),(0,0)), 'constant')
train_img_std_padded.shape

(11323, 16, 32, 54)

In [52]:
for lag in range(14):
    if lag==0:
        model = simple_unet_model(16,32,54)
        model.compile(loss='binary_crossentropy', optimizer='adam')
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=20, 
                                    verbose=1, mode='auto', restore_best_weights=True)
        model.fit(train_img_std_padded, y_train_img, validation_data=(val_img_std_padded,y_val_img),
                    callbacks=[monitor],epochs=100)
        t = model.predict(test_img_std_padded)

        ### t contains the predictions for each day and area

        ### Brier scores ### 
        print(f'All zeros Brier score {areas[i]}: {brier_score_loss(y_test_img.reshape(-1,1), np.zeros(len(y_test_img.reshape(-1,1))))}')
        print(f'Model Brier score 1: {brier_score_loss(y_test_img.reshape(-1,1), t.reshape(-1,1))}')
       
    else:
        model = simple_unet_model(16,32,54)
        model.compile(loss='binary_crossentropy', optimizer='adam')
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=20, 
                                    verbose=1, mode='auto', restore_best_weights=True)
        model.fit(train_img_std_padded[:-lag], y_train_img[lag:], validation_data=(val_img_std_padded[:-lag],y_val_img[lag:]),
                    callbacks=[monitor],epochs=100)
        t = model.predict(test_img_std_padded)
        ### Brier score ### 
        print(f'All zeros Brier score {brier_score_loss(y_test_img[lag:,:,:,:].reshape(-1,1), np.zeros(len(y_test_img[lag:,:,:,:].reshape(-1,1))))}')
        print(f'Model Brier score 1: {brier_score_loss(y_test_img_curr[lag:,:,:,:].reshape(-1,1), t[:-lag,:,:,:].reshape(-1,1))}')  

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 32, 54)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 16, 32, 16)   7792        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 16, 32, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 32, 16)   2320        ['dropout[0][0]']            

KeyboardInterrupt: 

In [3]:
import numpy as np
def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

In [18]:
compute_CI([46.020787954330444,50.140758991241455,44.16952180862427,43.06470513343811,45.58594989776611],5)

45.79634475708008 +- 2.1137846689672832


In [20]:
compute_CI([0.056430816650390625,0.02591848373413086,0.03510618209838867,0.03234386444091797,0.03227686882019043],5)

0.03641524314880371 +- 0.009161577984556524


In [21]:
compute_CI([0.4767742156982422,0.5182511806488037,0.42174863815307617,0.4476346969604492,0.4452531337738037],5)

0.461932373046875 +- 0.029043579982963356


In [22]:
compute_CI([0.6722421646118164,0.5140597820281982,0.4980301856994629,0.5369448661804199,0.4831843376159668],5)

0.5408922672271729 +- 0.05965065810051219


In [23]:
compute_CI([0.1438734531402588,0.1325550079345703,0.13147807121276855,0.12185502052307129,0.1191871166229248],5)

0.12978973388671874 +- 0.0076838353787733495
